In [94]:
import nltk
from textblob import TextBlob as tb
import math
import io
import codecs
from __future__ import division
import numpy
import pandas as pd
import matplotlib
%matplotlib inline 


In [124]:
import sys
import re
import os
import string
import re
os.chdir('/users/asheets/Documents/Work_Computer_new/Work_Computer/Grad_School/PREDICT_453/Notebooks/DSI/')

all_docs = []
blob_list = []
d = 24

cachedStopWords = nltk.corpus.stopwords.words('english')
pattern = re.compile(r'\b(' + r'|'.join(cachedStopWords) + r')\b\s*')

for i in range(16):
    doc_name = 'DSI' + str(d) + '.txt'
    try:
        with open(doc_name, 'r') as f:
            sample = f.read()
        sample = sample.decode('utf-8')
       #sample = sample.decode('ascii')
        sample = sample.lower()
        sample = re.sub(r'[^\w]', ' ', sample)
        sample = ''.join([i for i in sample if not i.isdigit()])
        sample = pattern.sub('', sample)
        sample = "".join(l for l in sample if l not in string.punctuation)
        sample2 = " ".join(k for k in tb(sample).noun_phrases)
        all_docs.append({'DSInum': d, 'raw_text': sample, 'noun_phrases_only':
        sample2})
        blob_list.append(tb(sample2))
        d = d + 1
    except IOError:
        d = d + 1
        pass
    
all_documents = pd.DataFrame(all_docs)
    
RTV = pd.read_csv('RTV.txt')


,DSInum,noun_phrases_only,raw_text
0,24,president trump executive order american energ...,president trump executive order american energ...
1,25,mr spicer good afternoon briefer side morning ...,mr spicer good afternoon everyone lot go...
2,26,report kremlin involvement donald trump electi...,bbc learned us officials verified key claim ...
3,27,temporary suspension entry nationals countries...,sec temporary suspension entry nationals c...
4,28,president donald trump executive order tuesday...,president donald trump signed executive order ...
5,29,internet service providers internet history se...,internet service providers may soon able sell ...
6,30,president donald trump wall u mexico border se...,president donald trump wall along u mexico bo...
7,31,trump administration plans press japan germany...,trump administration plans press japan german...
8,32,toledo ohio cream small business community lun...,toledo ohio cream small business community ...
9,33,donald trump tn budget thursday federal govern...,donald trump unveiled tn budget thursday far ...


In [118]:
#http://stevenloria.com/finding-important-words-in-a-document-using-tf-idf/

#computes "term frequency" which is the number of times a word appears in a document blob, 
#normalized by dividing by the total number of words in blob. 
def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

#returns the number of documents containing word. A generator expression is passed to the sum() function.
def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

#computes "inverse document frequency" which measures how common a word is among all documents in bloblist. 
#The more common a word is, the lower its idf. 
#We take the ratio of the total number of documents to the number of documents containing word, 
#then take the log of that. Add 1 to the divisor to prevent division by zero.
def idf(word, bloblist):
     return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

#computes the TF-IDF score. It is simply the product of  tf and idf.
def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [18]:
#compare all three articles
blob1 = tb(sample1)
blob2 = blob1.noun_phrases
term_freq = [tf(word,blob1) for word in blob1.words]
tf_df1 = pd.DataFrame({'word': blob1.words, 'doc1_term_freq': term_freq})

blob2 = tb(sample2)
term_freq = [tf(word,blob2) for word in blob2.words]
tf_df2 = pd.DataFrame({'word': blob2.words, 'doc2_term_freq': term_freq})

blob3 = tb(sample3)
term_freq = [tf(word,blob3) for word in blob3.words]
tf_df3 = pd.DataFrame({'word': blob3.words, 'doc3_term_freq': term_freq})

all_tf = pd.merge(pd.merge(tf_df1,tf_df2,on='word',how='outer'),tf_df3,on='word',how='outer').drop_duplicates()

bloblist = [blob1,blob2,blob3]

docs_containing1 = pd.DataFrame({'word': blob1.words, 'doc_freq': [n_containing(word,bloblist) for word in blob1.words]})
docs_containing2 = pd.DataFrame({'word': blob2.words, 'doc_freq': [n_containing(word,bloblist) for word in blob2.words]})
docs_containing3 = pd.DataFrame({'word': blob3.words, 'doc_freq': [n_containing(word,bloblist) for word in blob3.words]})

doc_freq = pd.concat([docs_containing1,docs_containing2,docs_containing3]).drop_duplicates().reset_index(drop=True)
df = pd.merge(doc_freq,all_tf,on='word',how='inner')
df = df.sort_values("doc_freq",ascending=False)

df['intermediate_calc'] = (len(bloblist) / df['doc_freq']).astype(float)
df['idf'] = df['intermediate_calc'].apply(math.log)
df = df.sort_values("idf",ascending=True)


scores1 = pd.DataFrame({'word': blob1.words, 'tf_idf_doc1': [tfidf(word, blob1, bloblist) for word in blob1.words]})
scores2 = pd.DataFrame({'word': blob2.words, 'tf_idf_doc2': [tfidf(word, blob2, bloblist) for word in blob2.words]})
scores3 = pd.DataFrame({'word': blob3.words, 'tf_idf_doc3': [tfidf(word, blob3, bloblist) for word in blob3.words]})

all_tfidf= pd.merge(pd.merge(scores1,scores2,on='word',how='outer'),scores3,on='word',how='outer').drop_duplicates()
all_tfidf = all_tfidf[['word', 'tf_idf_doc1', 'tf_idf_doc2', 'tf_idf_doc3']]
all_tfidf = all_tfidf.sort_values(by=['tf_idf_doc1', 'tf_idf_doc2','tf_idf_doc3'], ascending=[False,False,False])
all_tfidf.head(n=6)



,word,tf_idf_doc1,tf_idf_doc2,tf_idf_doc3
1981,future,0.014967,NaN,NaN
283,energy,0.011048,0.008275,NaN
782,creation,0.005987,NaN,NaN
805,economic,0.005987,NaN,NaN
809,companies,0.005987,NaN,NaN
811,workers,0.005987,NaN,NaN


In [19]:
##Compare just the two articles we know to be similar
two_tf = pd.merge(tf_df1,tf_df2,on='word',how='outer').drop_duplicates()

bloblist = [blob1,blob2]

docs_containing1 = pd.DataFrame({'word': blob1.words, 'doc_freq': [n_containing(word,bloblist) for word in blob1.words]})
docs_containing2 = pd.DataFrame({'word': blob2.words, 'doc_freq': [n_containing(word,bloblist) for word in blob2.words]})

doc_freq = pd.concat([docs_containing1,docs_containing2]).drop_duplicates().reset_index(drop=True)
df = pd.merge(doc_freq,two_tf,on='word',how='inner')
df = df.sort_values("doc_freq",ascending=False)

df['intermediate_calc'] = (len(bloblist) / df['doc_freq']).astype(float)
df['idf'] = df['intermediate_calc'].apply(math.log)
df = df.sort_values("idf",ascending=False)

scores1 = pd.DataFrame({'word': blob1.words, 'tf_idf_doc1': [tfidf(word, blob1, bloblist) for word in blob1.words]})
scores2 = pd.DataFrame({'word': blob2.words, 'tf_idf_doc2': [tfidf(word, blob2, bloblist) for word in blob2.words]})

two_tfidf= pd.merge(scores1,scores2,on='word',how='outer').drop_duplicates()
two_tfidf = two_tfidf[['word', 'tf_idf_doc1', 'tf_idf_doc2']]
two_tfidf = two_tfidf.sort_values(by=['tf_idf_doc1', 'tf_idf_doc2'], ascending=[False,False])
two_tfidf.head(n=6)


,word,tf_idf_doc1,tf_idf_doc2
501,future,0.009443,NaN
360,back,0.007555,NaN
242,new,0.005666,NaN
592,white,0.005666,NaN
595,house,0.005666,NaN
249,job,0.003777,NaN


In [ ]:
#compare all three using all pre-defined functions
bloblist2 = [blob1,blob2,blob3]
for i, blob in enumerate(bloblist2):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))


In [ ]:
#compare similar docs using all pre-defined functions
bloblist3 = [blob1,blob2]
for i, blob in enumerate(bloblist3):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

In [126]:
#compare all DSIs sing all pre-defined functions
DSI_list = all_documents["DSInum"]
for i, blob in enumerate(blob_list):
    print("Top words in document {}".format(DSI_list[i]))
    scores = {word: tfidf(word, blob, blob_list) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
        
        

Top words in document 24
	Word: energy, TF-IDF: 0.04677
	Word: future, TF-IDF: 0.02339
	Word: era, TF-IDF: 0.02328
	Word: coal, TF-IDF: 0.02328
	Word: power, TF-IDF: 0.01896
	Word: emissions, TF-IDF: 0.01871
	Word: carbon, TF-IDF: 0.01746
	Word: action, TF-IDF: 0.01746
	Word: executive, TF-IDF: 0.01534
	Word: environmental, TF-IDF: 0.01534
Top words in document 25
	Word: q, TF-IDF: 0.06141
	Word: mr, TF-IDF: 0.04138
	Word: spicer, TF-IDF: 0.0264
	Word: vote, TF-IDF: 0.01818
	Word: healthcare, TF-IDF: 0.01474
	Word: reform, TF-IDF: 0.01474
	Word: tax, TF-IDF: 0.01347
	Word: people, TF-IDF: 0.01317
	Word: american, TF-IDF: 0.01254
	Word: obamacare, TF-IDF: 0.01228
Top words in document 26
	Word: fbi, TF-IDF: 0.0429
	Word: russia, TF-IDF: 0.03786
	Word: steele, TF-IDF: 0.03544
	Word: intelligence, TF-IDF: 0.0284
	Word: russian, TF-IDF: 0.02577
	Word: cia, TF-IDF: 0.02255
	Word: co, TF-IDF: 0.01933
	Word: investigation, TF-IDF: 0.01767
	Word: voter, TF-IDF: 0.01611
	Word: kalugin, TF-IDF: 